In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [3]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [4]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'].isin([2013,2014]) == False]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [5]:
# Якутск Орджоникидзе, 56
sales_train.loc[sales_train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales_train.loc[sales_train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [6]:
max_sales=1000
sums = sales_train.groupby('item_id')['item_cnt_day'].sum().reset_index().rename(columns={"item_cnt_day":"item_total_sales"}).sort_values(by='item_total_sales')

ids_reject = sums[(sums['item_total_sales'] > 0) & (sums['item_total_sales'] < max_sales)]['item_id'].unique()

In [7]:
train_item_ids = sales_train['item_id'].unique()
train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))


In [8]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [97]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [98]:
ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [99]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')


In [100]:
a = training.groupby(['shop_id','date_block_num'])['shop_cnt_block'].sum()\
                        .reset_index()
a = a[a['shop_cnt_block'] == 0]
closed_shop_ids = a["shop_id"].unique()
closed_shop_ids = np.setdiff1d(closed_shop_ids, [36])
print(closed_shop_ids)


#with pd.option_context('display.max_rows', 999):
    #print(a[a['item_cnt_block'] == 0])

training = training[~training['shop_id'].isin(closed_shop_ids)]

[ 9 17 20 27 29 30 33 40 43 51 54]


In [101]:
training['item_cnt_block_mean'] = training.groupby(['item_id'])['item_cnt_block'].transform(np.mean)
training['item_cnt_block_min'] = training.groupby(['item_id'])['item_cnt_block'].transform(np.min)
training['item_cnt_block_max'] = training.groupby(['item_id'])['item_cnt_block'].transform(np.max)
training['item_cnt_block_std'] = training.groupby(['item_id'])['item_cnt_block'].transform(np.std)
training['item_cnt_block_med'] = training.groupby(['item_id'])['item_cnt_block'].transform(np.median)

In [102]:
training['shop_cnt_block_mean'] = training.groupby(['shop_id'])['shop_cnt_block'].transform(np.mean)
training['shop_cnt_block_min'] = training.groupby(['shop_id'])['shop_cnt_block'].transform(np.min)
training['shop_cnt_block_max'] = training.groupby(['shop_id'])['shop_cnt_block'].transform(np.max)
training['shop_cnt_block_std'] = training.groupby(['shop_id'])['shop_cnt_block'].transform(np.std)
training['shop_cnt_block_med'] = training.groupby(['shop_id'])['shop_cnt_block'].transform(np.median)

In [103]:
training['category_cnt_block_mean'] = training.groupby(['item_category_id'])['category_cnt_block'].transform(np.mean)
training['category_cnt_block_min'] = training.groupby(['item_category_id'])['category_cnt_block'].transform(np.min)
training['category_cnt_block_max'] = training.groupby(['item_category_id'])['category_cnt_block'].transform(np.max)
training['category_cnt_block_std'] = training.groupby(['item_category_id'])['category_cnt_block'].transform(np.std)
training['category_cnt_block_med'] = training.groupby(['item_category_id'])['category_cnt_block'].transform(np.median)

In [104]:
training['shop_category_cnt_block_mean'] = training.groupby(['shop_id', 'item_category_id'])['shop_category_cnt_block'].transform(np.mean)
training['shop_category_cnt_block_min'] = training.groupby(['shop_id', 'item_category_id'])['shop_category_cnt_block'].transform(np.min)
training['shop_category_cnt_block_max'] = training.groupby(['shop_id', 'item_category_id'])['shop_category_cnt_block'].transform(np.max)
training['shop_category_cnt_block_std'] = training.groupby(['shop_id', 'item_category_id'])['shop_category_cnt_block'].transform(np.std)
training['shop_category_cnt_block_med'] = training.groupby(['shop_id', 'item_category_id'])['shop_category_cnt_block'].transform(np.median)

In [105]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,item_cnt_block_mean,item_cnt_block_min,item_cnt_block_max,item_cnt_block_std,item_cnt_block_med,shop_cnt_block_mean,shop_cnt_block_min,shop_cnt_block_max,shop_cnt_block_std,shop_cnt_block_med,category_cnt_block_mean,category_cnt_block_min,category_cnt_block_max,category_cnt_block_std,category_cnt_block_med,shop_category_cnt_block_mean,shop_category_cnt_block_min,shop_category_cnt_block_max,shop_category_cnt_block_std,shop_category_cnt_block_med
1384611,11430,31,25,0,40,1.0,7404.0,12100.0,1787.0,2,1.5,0.0,5.0,1.501789,1.0,6765.9,5714.0,8815.0,893.551843,6416.0,10174.3,6779.0,15109.0,2444.994580,9293.5,1563.2,1079.0,2074.0,301.291137,1502.5
2501441,20352,44,25,0,72,0.0,917.0,1875.0,31.0,2,1.2,0.0,11.0,3.284156,0.0,807.3,619.0,932.0,101.018857,799.5,1557.9,1122.0,2432.0,404.149620,1367.0,13.3,5.0,31.0,8.027675,9.5
262660,2428,38,24,0,31,0.0,1441.0,4824.0,0.0,1,0.0,0.0,0.0,0.000000,0.0,1371.3,1110.0,1781.0,184.305382,1349.5,1575.7,688.0,4824.0,1200.883307,1097.0,0.0,0.0,0.0,0.000000,0.0
1683189,13581,50,33,0,61,4.0,949.0,2290.0,13.0,10,0.4,0.0,4.0,1.201431,0.0,1040.0,895.0,1437.0,154.358886,1016.5,787.1,446.0,2290.0,515.823669,654.0,7.7,3.0,13.0,2.610457,8.0
1930800,15414,3,24,0,63,0.0,977.0,2330.0,9.0,1,0.2,0.0,2.0,0.600716,0.0,715.8,535.0,977.0,108.940314,734.5,1880.9,1316.0,2555.0,408.559572,1932.0,14.3,9.0,18.0,3.002813,15.0
1843822,14958,55,26,0,47,0.0,2769.0,18.0,0.0,3,6.6,0.0,66.0,19.823614,0.0,3167.0,1658.0,6867.0,1809.497721,2334.0,687.6,0.0,3421.0,1018.505482,149.0,0.0,0.0,0.0,0.000000,0.0
921773,7543,15,27,0,30,10.0,1364.0,14751.0,334.0,4,5.3,0.0,12.0,4.153441,4.5,1411.7,1243.0,2046.0,224.641929,1346.0,7487.0,4150.0,14751.0,2860.257720,6519.5,201.2,114.0,334.0,58.803933,185.5
1726247,13818,5,31,0,37,10.0,1294.0,3426.0,44.0,8,4.5,0.0,14.0,5.302541,1.0,1106.3,954.0,1404.0,132.350113,1073.0,4138.5,2989.0,5959.0,918.820506,3908.5,50.2,32.0,74.0,15.428283,48.0
2116334,16534,6,28,0,40,6.0,1748.0,9208.0,172.0,5,7.7,2.0,21.0,5.164640,6.5,1850.9,1484.0,2329.0,288.380505,1775.0,10174.3,6779.0,15109.0,2444.994580,9293.5,226.4,166.0,308.0,54.673835,207.0
455919,3759,16,33,0,20,1.0,963.0,6596.0,110.0,10,0.1,0.0,1.0,0.300358,0.0,1193.1,963.0,1698.0,195.685560,1150.5,5422.5,4034.0,7130.0,1055.245565,4942.0,86.3,48.0,111.0,18.672560,83.5


In [239]:
training.describe()

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,item_cnt_block_units,item_cnt_block_median,item_cnt_block_mean,item_cnt_block_max,item_cnt_block_min,item_cnt_block_std,shop_cnt_block_units,shop_cnt_block_median,shop_cnt_block_mean,shop_cnt_block_max,shop_cnt_block_min,shop_cnt_block_std,category_cnt_block_units,category_cnt_block_median,category_cnt_block_mean,category_cnt_block_max,category_cnt_block_min,category_cnt_block_std,shop_category_cnt_block_units,shop_category_cnt_block_median,shop_category_cnt_block_mean,shop_category_cnt_block_max,shop_category_cnt_block_min,shop_category_cnt_block_std
count,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06,2.097560e+06
mean,1.101830e+04,3.156098e+01,2.850000e+01,2.445384e-01,4.627815e+01,1.257451e+01,1.699154e+03,3.773882e+03,8.389024e+01,5.500000e+00,5.155550e+03,8.791634e+00,1.257451e+01,4.066224e+01,3.116302e+00,1.149800e+01,8.692870e+07,1.633695e+03,1.699154e+03,2.342293e+03,1.339854e+03,3.034243e+02,8.311411e+08,3.444695e+03,3.773882e+03,5.851549e+03,2.520990e+03,9.875358e+02,4.513030e+05,8.002772e+01,8.389024e+01,1.289736e+02,5.351564e+01,2.269051e+01
std,6.246577e+03,1.773250e+01,2.872282e+00,1.080989e+00,1.676899e+01,8.034849e+01,1.394110e+03,3.751384e+03,1.724458e+02,2.872282e+00,2.534712e+04,5.266674e+01,6.182224e+01,1.920728e+02,2.927392e+01,5.008119e+01,6.728722e+07,1.260525e+03,1.315231e+03,1.902227e+03,1.074559e+03,3.487790e+02,1.170224e+09,3.245611e+03,3.522610e+03,5.108302e+03,2.405506e+03,8.299934e+02,1.143970e+06,1.520227e+02,1.616670e+02,2.734789e+02,1.077633e+02,5.558359e+01
min,3.000000e+01,2.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,-1.000000e+00,3.830000e+02,-1.000000e+00,-1.000000e+00,1.000000e+00,-4.100000e+01,0.000000e+00,-1.000000e-01,0.000000e+00,-1.000000e+00,0.000000e+00,2.178904e+07,4.220000e+02,4.259000e+02,4.840000e+02,3.830000e+02,3.450961e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,-4.000000e+00,0.000000e+00,-1.000000e-01,0.000000e+00,-1.000000e+00,0.000000e+00
25%,5.394250e+03,1.600000e+01,2.600000e+01,0.000000e+00,3.700000e+01,0.000000e+00,9.320000e+02,6.800000e+02,4.000000e+00,3.000000e+00,4.100000e+02,0.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00,1.184662e+00,5.169718e+07,9.890000e+02,1.010500e+03,1.247000e+03,7.900000e+02,1.242178e+02,3.709315e+07,6.540000e+02,7.871000e+02,1.622000e+03,4.460000e+02,3.046296e+02,4.023000e+03,3.500000e+00,4.600000e+00,1.200000e+01,0.000000e+00,3.130728e+00
50%,1.119050e+04,3.400000e+01,2.850000e+01,0.000000e+00,4.300000e+01,2.000000e+00,1.272000e+03,2.203000e+03,3.300000e+01,5.500000e+00,1.722000e+03,2.000000e+00,4.200000e+00,1.100000e+01,0.000000e+00,3.138314e+00,6.666660e+07,1.318500e+03,1.303100e+03,1.715000e+03,9.900000e+02,1.843054e+02,1.802443e+08,1.917000e+03,1.880900e+03,4.824000e+03,1.316000e+03,9.188206e+02,4.686500e+04,3.150000e+01,3.400000e+01,6.000000e+01,1.700000e+01,1.218249e+01
75%,1.606725e+04,4.700000e+01,3.100000e+01,0.000000e+00,5.800000e+01,1.000000e+01,1.740000e+03,6.353000e+03,1.020000e+02,8.000000e+00,4.756000e+03,8.500000e+00,1.160000e+01,3.000000e+01,2.000000e+00,8.594192e+00,9.245635e+07,1.699500e+03,1.807200e+03,2.329000e+03,1.320000e+03,2.777342e+02,1.899903e+09,6.413500e+03,6.895700e+03,9.291000e+03,4.913000e+03,1.348041e+03,4.928000e+05,1.015000e+02,1.043000e+02,1.530000e+02,6.800000e+01,2.467247e+01
max,2.216700e+04,5.900000e+01,3.300000e+01,2.000000e+01,8.300000e+01,7.869000e+03,8.815000e+03,1.510900e+04,5.129000e+03,1.000000e+01,1.582846e+06,3.512000e+03

In [106]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'item_cnt_block', 'shop_cnt_block',
       'category_cnt_block', 'shop_category_cnt_block', 'month',
       'item_cnt_block_mean', 'item_cnt_block_min', 'item_cnt_block_max',
       'item_cnt_block_std', 'item_cnt_block_med', 'shop_cnt_block_mean',
       'shop_cnt_block_min', 'shop_cnt_block_max', 'shop_cnt_block_std',
       'shop_cnt_block_med', 'category_cnt_block_mean',
       'category_cnt_block_min', 'category_cnt_block_max',
       'category_cnt_block_std', 'category_cnt_block_med',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_min',
       'shop_category_cnt_block_max', 'shop_category_cnt_block_std',
       'shop_category_cnt_block_med'], dtype=object)

In [107]:
features = [
    
    'item_cnt_block', 'shop_cnt_block',
       'category_cnt_block', 'shop_category_cnt_block',
       'item_cnt_block_mean', 'item_cnt_block_min', 'item_cnt_block_max',
       'item_cnt_block_std', 'item_cnt_block_med', 'shop_cnt_block_mean',
       'shop_cnt_block_min', 'shop_cnt_block_max', 'shop_cnt_block_std',
       'shop_cnt_block_med', 'category_cnt_block_mean',
       'category_cnt_block_min', 'category_cnt_block_max',
       'category_cnt_block_std', 'category_cnt_block_med',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_min',
       'shop_category_cnt_block_max', 'shop_category_cnt_block_std',
       'shop_category_cnt_block_med'

]

In [108]:
training.fillna(0,inplace=True)

In [111]:
training[features].sample(10)

,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_cnt_block_mean,item_cnt_block_min,item_cnt_block_max,item_cnt_block_std,item_cnt_block_med,shop_cnt_block_mean,shop_cnt_block_min,shop_cnt_block_max,shop_cnt_block_std,shop_cnt_block_med,category_cnt_block_mean,category_cnt_block_min,category_cnt_block_max,category_cnt_block_std,category_cnt_block_med,shop_category_cnt_block_mean,shop_category_cnt_block_min,shop_category_cnt_block_max,shop_category_cnt_block_std,shop_category_cnt_block_med
2347228,-0.131608,-0.717832,0.801069,-0.140076,-0.101493,-0.038133,-0.159639,-0.167612,-0.043512,-0.686657,-0.634459,-0.690211,-0.569641,-0.684832,1.816954,1.770110,1.812237,1.755989,1.802066,0.001053,0.053927,-0.063845,-0.060180,-0.004129
2411310,-0.144054,-0.246647,-0.872713,-0.479565,-0.161342,-0.106453,-0.185670,-0.199666,-0.109968,-0.612869,-0.580813,-0.514820,-0.341784,-0.629718,-0.976033,-0.969855,-1.045465,-1.050711,-0.969215,-0.511359,-0.489222,-0.466414,-0.404901,-0.518408
2112055,-0.156500,-0.610744,-0.004767,-0.169342,-0.179135,-0.106453,-0.133607,-0.139623,-0.166930,-0.616041,-0.537341,-0.686535,-0.638572,-0.600192,0.103508,0.194558,0.021034,-0.082790,0.142902,-0.286047,-0.367482,-0.270669,-0.174058,-0.259610
1575260,-0.156500,0.275941,-1.005997,-0.479565,0.128198,-0.106453,0.220426,0.378461,-0.166930,-0.187137,-0.060077,-0.130430,-0.213622,-0.195890,-0.876135,-1.048009,-0.475804,0.037313,-1.015432,-0.490138,-0.489222,-0.411014,-0.317205,-0.515090
1970301,-0.106717,-1.184734,2.219479,-0.479565,-0.140314,-0.106453,-0.170051,-0.190157,-0.090980,-1.228394,-1.209767,-1.031540,-0.576738,-1.255658,1.816954,1.770110,1.812237,1.755989,1.802066,-0.495131,-0.489222,-0.370388,-0.263047,-0.518408
46486,-0.156500,-0.700698,-0.996401,-0.479565,-0.198545,-0.106453,-0.206496,-0.220426,-0.166930,-0.643608,-0.637234,-0.715417,-0.570906,-0.626175,-1.052567,-1.035537,-1.123965,-1.161723,-1.041621,-0.511359,-0.489222,-0.466414,-0.404901,-0.518408
992169,-0.156500,-0.665716,1.041247,-0.046424,-0.201780,-0.106453,-0.206496,-0.223590,-0.166930,-0.616041,-0.537341,-0.686535,-0.638572,-0.600192,1.816954,1.770110,1.812237,1.755989,1.802066,-0.096935,-0.058449,-0.159870,-0.201864,-0.067170
1493124,-0.106717,-0.337314,0.687511,0.193560,-0.128991,-0.038133,-0.175258,-0.194730,-0.090980,-0.352234,-0.319057,-0.313173,-0.297309,-0.349816,0.886223,0.994390,0.673306,0.434348,0.914714,0.139610,-0.030355,0.032181,0.028538,0.214853
928044,-0.156500,-0.662860,-0.764487,-0.456152,-0.203398,-0.106453,-0.211702,-0.229587,-0.166930,-0.712713,-0.714928,-0.691786,-0.548012,-0.677352,-0.737715,-0.715023,-0.712673,-0.678338,-0.756004,-0.474535,-0.479857,-0.414708,-0.340607,-0.481910
1739997,0.055079,-0.512224,0.599277,-0.438592,-0.146784,-0.106453,-0.123194,-0.106592,-0.166930,-0.616041,-0.537341,-0.686535,-0.638572,-0.600192,0.886223,0.994390,0.673306,0.434348,0.914714,-0.418987,-0.489222,-0.100778,0.109157,-0.468639


In [110]:
from sklearn import preprocessing


training[features] = preprocessing.scale(training[features])


In [112]:
a = training[['shop_item_cnt_block'] + ['date_block_num','item_id', 'shop_id'] + features].sort_values(by=["date_block_num"]).groupby(["item_id", "shop_id"])#['item_id_mean_encoding']#[features+['item_cnt_block']]

In [113]:
#groups
lstm_data = []
lstm_y = []

for name, group in a:
    #print(group.values)
    steps = []
    ys = []
    #print("group.values",group.values)
    for step in group.values:
        #print("step", len(step))
        #step is np.array 
        #step[4:] is np.array print(type(step[4:]))
        steps.append(step[4:])
        #print(step[9])
        #print(step[0])
        ys.append(step[0])
    #remove last
    #print(type(steps[0:8]))
    lstm_data.append(np.array(steps[0:8]))
    #remove first
    #print(ys)
    lstm_y.append(ys[1:9][-1])
    
lstm_y = np.array([np.array([y]) for y in lstm_y])

In [114]:
small_data = np.array(lstm_data)
small_y = np.array(lstm_y)

In [115]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(small_data, small_y, test_size=0.20, random_state=42)

In [116]:
features

['item_cnt_block',
 'shop_cnt_block',
 'category_cnt_block',
 'shop_category_cnt_block',
 'item_cnt_block_mean',
 'item_cnt_block_min',
 'item_cnt_block_max',
 'item_cnt_block_std',
 'item_cnt_block_med',
 'shop_cnt_block_mean',
 'shop_cnt_block_min',
 'shop_cnt_block_max',
 'shop_cnt_block_std',
 'shop_cnt_block_med',
 'category_cnt_block_mean',
 'category_cnt_block_min',
 'category_cnt_block_max',
 'category_cnt_block_std',
 'category_cnt_block_med',
 'shop_category_cnt_block_mean',
 'shop_category_cnt_block_min',
 'shop_category_cnt_block_max',
 'shop_category_cnt_block_std',
 'shop_category_cnt_block_med']

In [118]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


#x_train_scaled = MinMaxScaler().fit_transform(x_train[features])


#x_reshaped = np.reshape(x_train_scaled, (x_train_scaled.shape[0], 10, x_train_scaled.shape[1]))
    
#x_val_scaled_reshaped = np.reshape(x_val_scaled, (x_val_scaled.shape[0], 1, x_val_scaled.shape[1]))


callbacks = [
    EarlyStopping(monitor='val_loss', patience=1, verbose=0)
]


my_model = Sequential()
my_model.add(LSTM(units = 16,input_shape = (8,len(features))))
my_model.add(Dropout(0.5))
my_model.add(BatchNormalization())
my_model.add(Dropout(0.5))
#my_model.add(BatchNormalization())
#my_model.add(LSTM(units = 16))
#my_model.add(BatchNormalization())
#my_model.add(Flatten())
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()


#lstmd_dataa = np.array(np.array(lstm_data)).reshape(271148,8,51)
#print(lstmd_dataa.shape)
#print(lstmd_dataa)
#lstm_yy = np.array([np.array([y]) for y in lstm_y[0:100]])
#print(lstm_yy.shape)
#print(lstm_yy)

history = my_model.fit(train_data, y_train, batch_size=4096, epochs=100,
                      validation_data=(val_data,y_val), callbacks=callbacks
                      )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 16)                2624      
_________________________________________________________________
dropout_11 (Dropout)         (None, 16)                0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 16)                64        
_________________________________________________________________
dropout_12 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 2,705
Trainable params: 2,673
Non-trainable params: 32
_________________________________________________________________
Train on 171897 samples, validate on 42975 samples
Epoch 1/100
171897/171897 [==============================] - 7s 43us/step - loss: 2.9979

In [68]:
features

['item_cnt_block',
 'shop_cnt_block',
 'category_cnt_block',
 'shop_category_cnt_block',
 'item_cnt_block_mean',
 'item_cnt_block_max',
 'item_cnt_block_std',
 'shop_cnt_block_mean',
 'shop_cnt_block_max',
 'shop_cnt_block_std',
 'category_cnt_block_mean',
 'category_cnt_block_max',
 'category_cnt_block_std',
 'shop_category_cnt_block_mean',
 'shop_category_cnt_block_max',
 'shop_category_cnt_block_std']

In [70]:
lstm_data[0]

array([[ 0.01774133, -0.51083057,  3.02158358, -0.15593446, -0.10634544,
        -0.13881323, -0.14571611, -0.65034494, -0.71247718, -0.6474541 ,
         1.8169537 ,  1.81223685,  1.75598899, -0.10818685, -0.16810644,
        -0.20735544],
       [ 0.01774133, -0.56821488,  2.21947946, -0.09214629, -0.10634544,
        -0.13881323, -0.14571611, -0.65034494, -0.71247718, -0.6474541 ,
         1.8169537 ,  1.81223685,  1.75598899, -0.10818685, -0.16810644,
        -0.20735544],
       [-0.09427076, -0.67222394,  2.4817827 , -0.09214629, -0.10634544,
        -0.13881323, -0.14571611, -0.65034494, -0.71247718, -0.6474541 ,
         1.8169537 ,  1.81223685,  1.75598899, -0.10818685, -0.16810644,
        -0.20735544],
       [-0.10671655, -0.60264547,  1.84175212, -0.00516242, -0.10634544,
        -0.13881323, -0.14571611, -0.65034494, -0.71247718, -0.6474541 ,
         1.8169537 ,  1.81223685,  1.75598899, -0.10818685, -0.16810644,
        -0.20735544],
       [-0.09427076, -0.61412233,  1

In [119]:
training_test = training[(training['shop_id'].isin(test['shop_id'].unique())) & (training['item_id'].isin(test['item_id'].unique()))]

In [120]:
test_lstm_data = []

for index, row in test.iterrows():
    if index % 10000 == 0:
        print(index)
    line = training_test[(training_test['shop_id'] == row['shop_id']) & (training_test['item_id'] == row['item_id'])].sort_values(by=['date_block_num'])[features].values
    test_lstm_data.append(line[2:11])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000


In [92]:
training_test[(training_test['shop_id'] == 5) & (training_test['item_id'] == 5037)].sort_values(by=['date_block_num'])

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,mean_item_cnt_block,item_cnt_block_mean,item_cnt_block_min,item_cnt_block_max,item_cnt_block_std,item_cnt_block_med,shop_cnt_block_mean,shop_cnt_block_min,shop_cnt_block_max,shop_cnt_block_std,shop_cnt_block_med,shop_category_cnt_block_mean,shop_category_cnt_block_min,shop_category_cnt_block_max,shop_category_cnt_block_std,shop_category_cnt_block_med
633910,5037,5,24,2,19,1.187645,-0.182394,0.703505,0.152587,-1.566699,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673
633911,5037,5,25,0,19,0.266657,-0.398711,0.110657,-0.052277,-1.218544,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673
633912,5037,5,26,0,19,0.229320,-0.393000,-0.009832,-0.122516,-0.870388,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673
633913,5037,5,27,0,19,0.191982,-0.432265,-0.265204,-0.204462,-0.522233,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673
633914,5037,5,28,1,19,0.926284,-0.462250,-0.101798,-0.046424,-0.174078,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673
633915,5037,5,29,1,19,1.150308,-0.503657,-0.076474,-0.063983,0.174078,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673
633916,5037,5,30,1,19,0.515573,-0.477242,-0.158843,-0.104956,0.522233,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673
633917,5037,5,31,3,19,1.324549,-0.260925,0.238077,0.322332,0.870388,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673
633918,5037,5,32,1,19,1.212537,-0.405136,0.054678,0.164294,1.218544,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673
633919,5037,5,33,0,19,0.154645,-0.433693,-0.354771,-0.040570,1.566699,0.930499,0.930499,0.747549,0.407855,0.513644,1.171677,-0.417789,-0.327382,-0.467559,-0.480355,-0.410836,0.000429,-0.049084,0.039568,0.069131,-0.030673


In [81]:
preds = my_model.predict(test_lstm_data[0:100],batch_size=len(test_lstm_data))

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 100 arrays: [array([[[ 2.29319727e-01, -3.92999669e-01, -9.83155196e-03,
         -1.22516063e-01, -8.70388280e-01,  9.30498548e-01,
          9.30498548e-01,  7.47549446e-01,  4.07854704e-01,
          5.1364392...

In [84]:
np.array(test_lstm_data).shape

(214200, 1, 8, 21)

In [90]:
test_lstm_data[0]

array([[ 2.29319727e-01, -3.92999669e-01, -9.83155196e-03,
        -1.22516063e-01, -8.70388280e-01,  9.30498548e-01,
         9.30498548e-01,  7.47549446e-01,  4.07854704e-01,
         5.13643920e-01,  1.17167656e+00, -4.17789237e-01,
        -3.27381819e-01, -4.67558958e-01, -4.80355462e-01,
        -4.10835523e-01,  4.29161459e-04, -4.90843041e-02,
         3.95676025e-02,  6.91307747e-02, -3.06728469e-02],
       [ 1.91982363e-01, -4.32265130e-01, -2.65204019e-01,
        -2.04461758e-01, -5.22232968e-01,  9.30498548e-01,
         9.30498548e-01,  7.47549446e-01,  4.07854704e-01,
         5.13643920e-01,  1.17167656e+00, -4.17789237e-01,
        -3.27381819e-01, -4.67558958e-01, -4.80355462e-01,
        -4.10835523e-01,  4.29161459e-04, -4.90843041e-02,
         3.95676025e-02,  6.91307747e-02, -3.06728469e-02],
       [ 9.26283860e-01, -4.62249663e-01, -1.01797628e-01,
        -4.64236325e-02, -1.74077656e-01,  9.30498548e-01,
         9.30498548e-01,  7.47549446e-01,  4.07854704e

In [84]:
x[2:11].reshape(1,8,16)

array([[[ 0.22931973, -0.4233195 , -0.00983155, -0.13273877,
          0.93049855,  0.4078547 ,  0.51364392, -0.45076019,
         -0.49326033, -0.49049451,  0.01490886,  0.10990946,
          0.09066831, -0.01169217,  0.02934944,  0.0607451 ],
        [ 0.19198236, -0.46277121, -0.26520402, -0.21392371,
          0.93049855,  0.4078547 ,  0.51364392, -0.45076019,
         -0.49326033, -0.49049451,  0.01490886,  0.10990946,
          0.09066831, -0.01169217,  0.02934944,  0.0607451 ],
        [ 0.92628386, -0.49289797, -0.10179763, -0.05735274,
          0.93049855,  0.4078547 ,  0.51364392, -0.45076019,
         -0.49326033, -0.49049451,  0.01490886,  0.10990946,
          0.09066831, -0.01169217,  0.02934944,  0.0607451 ],
        [ 1.15030805, -0.5345016 , -0.07647364, -0.07474952,
          0.93049855,  0.4078547 ,  0.51364392, -0.45076019,
         -0.49326033, -0.49049451,  0.01490886,  0.10990946,
          0.09066831, -0.01169217,  0.02934944,  0.0607451 ],
        [ 0.51557285

In [ ]:
training[['shop_item_cnt_block'] + ['date_block_num','item_id', 'shop_id'] + features].sort_values(by=["date_block_num"]).groupby(["item_id", "shop_id"])#['item_id_mean_encoding']#[features+['item_cnt_block']]

In [ ]:
features = [
    
'item_id_mean_encoding',
       'shop_id_mean_encoding', #'item_category_id_mean_encoding',
#       'month_mean_encoding', 'shop_cat_mean_encoding',
#       'shop_item_mean_encoding', 'date_block_num_mean_encoding',
 
#       'item_block_units_rolling_3', 'item_block_mean_rolling_3',
#       'item_block_median_rolling_3', 'item_block_min_rolling_3',
#       'item_block_max_rolling_3', 'item_block_std_rolling_3',
    
#       'shop_block_units_rolling_3', 'shop_block_mean_rolling_3',
#       'shop_block_median_rolling_3', 'shop_block_min_rolling_3',
#       'shop_block_max_rolling_3', 'shop_block_std_rolling_3',
    
#       'cat_block_units_rolling_3', 'cat_block_mean_rolling_3',
#       'cat_block_median_rolling_3', 'cat_block_min_rolling_3',
#       'cat_block_max_rolling_3', 'cat_block_std_rolling_3',
    
       'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3',
       'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3',
       'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3',
    
    
#       'item_block_units_lag_1', 'item_block_mean_lag_1',
#       'item_block_median_lag_1', 'item_block_min_lag_1',
#       'item_block_max_lag_1', 'item_block_std_lag_1',
    
#       'shop_block_units_lag_1', 'shop_block_mean_lag_1',
#       'shop_block_median_lag_1', 'shop_block_min_lag_1',
       'shop_block_max_lag_1', 'shop_block_std_lag_1',
    
#      'cat_block_units_lag_1', 'cat_block_mean_lag_1',
#       'cat_block_median_lag_1', 'cat_block_min_lag_1',
#       'cat_block_max_lag_1', 'cat_block_std_lag_1',
    
       'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1',
#       'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1',
#       'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1',
    
       'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1',
#       'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1',
#       'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1',
    
       'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1',
    
    'shop_pred', 'shop_cat_pred', 'cat_pred'

]


In [60]:
features = ['item_id_mean_encoding',
 'shop_cat_mean_encoding',
 'shop_item_block_units_lag_1',
 'shop_cat_pred',
 'shop_cat_block_units_lag_1',
 'cat_pred',
 'comp2_lag_1',
 'shop_cat_block_max_rolling_3',
 'shop_block_mean_lag_1',
 'item_share_block_lag_1',
 'shop_item_block_mean_lag_1']

In [40]:
from sklearn.preprocessing import MinMaxScaler

x_train_scaled = MinMaxScaler().fit_transform(x_train[features])
x_val_scaled = MinMaxScaler().fit_transform(x_val[features])

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [63]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor


lr_model =  BayesianRidge()
lr_model.fit(x_train_scaled, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_scaled)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

rmse:  1.0587979027152195


In [ ]:
lr_model.coef_

In [45]:
#test            = pd.read_csv('test.csv.gz')
test = test.set_index('item_id').join(items.set_index('item_id'))
test.reset_index(inplace=True)

In [46]:
def predict(shop_id):
    return shop_models[shop_id].predict([[34]])[0][0]

#test['shop_pred'] = test.apply(lambda row: predict(row['shop_id']), axis=1)

def predict(shop_id, cat_id):
    if shop_id in shop_cat_models and cat_id in shop_cat_models[shop_id]:
        return shop_cat_models[shop_id][cat_id].predict([[34]])[0][0]

test['shop_cat_pred'] = test.apply(lambda row: predict(row['shop_id'],row['item_category_id']), axis=1)

def predict(cat_id):
    if cat_id in cat_models:
        return cat_models[cat_id].predict([[34]])[0][0]

test['cat_pred'] = test.apply(lambda row: predict(row['item_category_id']), axis=1)

In [47]:
item_features = [ 
    'item_id_mean_encoding'
                ]

merge_col = ['item_id']
cols=item_features+merge_col

test = test.merge(training.drop_duplicates('item_id')[cols], on=merge_col, how='left')

In [ ]:
shop_features = [
        'shop_id_mean_encoding'
]

merge_col = ['shop_id']
cols=shop_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [ ]:
cat_features = [
        'item_category_id_mean_encoding'#,'cat_me_real'
]

merge_col = ['item_category_id']
cols=cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [ ]:
shop_item_features = [
        'shop_item_mean_encoding'
]

merge_col = ['shop_id', 'item_id']
cols=shop_item_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [53]:
shop_cat_features = [
        'shop_cat_mean_encoding'
]

merge_col = ['shop_id', 'item_id']
cols=shop_cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [48]:
def add_rolls_test(df, cols, name, rolls = [3]):
    for roll in rolls:
        print(name, roll)
        roll_name = name+"_rolling_" + str(roll)
        roll_name_tmp = roll_name + "_tmp"
        
        try:
            df.drop(columns=[roll_name],inplace=True)
        except:
            pass       

    
        block_units_rolling_temp = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].rolling(roll,min_periods=2).mean().reset_index()\
            .rename(columns={name:roll_name})\
            [cols+[roll_name]]
        
        print([cols[0:len(cols)-1]+[roll_name]])
        thirty_three = block_units_rolling_temp[block_units_rolling_temp['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1]+[roll_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')
    

        del block_units_rolling_temp
        gc.collect()
        

    
    return df
    

#test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_mean')
#test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_units')
#test = add_rolls_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')

#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_median')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_min')
test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_max')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_std')

shop_cat_block_max 3
[['shop_id', 'item_category_id', 'shop_cat_block_max_rolling_3']]


In [ ]:
test = add_rolls_test(test, ['item_category_id','date_block_num'], 'cat_block_mean')

In [ ]:
test = add_rolls_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

In [49]:
def add_lags_test(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()
        
        thirty_three = result[result['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1] + [lag_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')

        gc.collect()
    
    return df
                                         

                                        
#test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_mean')
#test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_units')
test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
#test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')

#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_max')
#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_std')

test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')

test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

test = add_lags_test(test, ['item_id','date_block_num'], 'item_share_block')
#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_share_block')
#test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'comp2')

shop_block_mean 1
shop_cat_block_units 1
shop_item_block_units 1
shop_item_block_mean 1
item_share_block 1


In [ ]:
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')
#test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')

In [50]:
#training['comp2'] = training['item_share_block'] * training['shop_share_block']


test = add_lags_test(test, ['item_id','shop_id','date_block_num'], 'comp2')

comp2 1


In [54]:
test.fillna(0, inplace=True)

In [55]:
from sklearn.preprocessing import MinMaxScaler

test_scaled = MinMaxScaler().fit_transform(test[features])


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [64]:
preds = lr_model.predict(test_scaled)
preds.clip(0,20,out=preds)


array([ 0.        ,  0.        ,  0.15157035, ...,  0.26322811,
        0.28951454,  0.21382811])

In [92]:
preds = lg_model.predict(test[features])
preds.clip(0,20,out=preds)


array([ 0.11804522,  0.07073731,  0.18383498, ...,  0.04995914,
        0.28776269,  0.09947505])

In [96]:

print(np.mean(preds))
print(np.max(preds))

0.291327336924
17.0728554221


In [ ]:
preds = np.mean(np.array([lg_preds, lr_preds]),axis=0)

In [97]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

In [95]:
lr_preds = pd.read_csv('lr110.csv')['item_cnt_month']
lg_preds = pd.read_csv('lg110.csv')['item_cnt_month']
#cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']


#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

preds = (lg_preds * 0.50) + (lr_preds * 0.50)

In [ ]:
training.columns.values

In [30]:
all_features = ['item_id_mean_encoding',
       'shop_id_mean_encoding', 'item_category_id_mean_encoding',
       'month_mean_encoding', 'shop_cat_mean_encoding',
       'shop_item_mean_encoding', 'date_block_num_mean_encoding',
       'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3',
       'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3',
       'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3',
       'shop_block_units_lag_1', 'shop_block_mean_lag_1',
       'shop_block_median_lag_1', 'shop_block_min_lag_1',
       'shop_block_max_lag_1', 'shop_block_std_lag_1',
       'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1',
       'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1',
       'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1',
       'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1',
       'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1',
       'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1',
       'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1',
       'shop_share', 'item_share', 'shop_pred', 'shop_cat_pred',
       'cat_pred']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Create a random forest classifier
clf = RandomForestRegressor(n_estimators=10, random_state=0, n_jobs=8)

# Train the classifier
clf.fit(x_train[all_features], y_train)

# Print the name and gini importance of each feature


In [ ]:
f_ = sorted(list(zip(all_features, clf.feature_importances_)), key=lambda x: x[1])[::-1][0:]

features = [f[0] for f in f_]

from sklearn.preprocessing import MinMaxScaler

x_train_scaled = MinMaxScaler().fit_transform(x_train[features])
x_val_scaled = MinMaxScaler().fit_transform(x_val[features])


lr_model =  Ridget(alpha=0)
lr_model.fit(x_train_scaled, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_scaled)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

In [ ]:
sorted(list(zip(all_features, clf.feature_importances_)), key=lambda x: x[1])[::-1]

In [ ]:
all_features

In [38]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor
import math

all_f = all_features.copy()
keep = []

print(all_f)

best = 10000
while len(all_f) > 0:
    print("keep", keep)
    len_keep = len(keep)
    #print(len(all_f))
    res = {}
    for idx,feat in enumerate(all_f):

        potential = keep + [feat]

        x_train_scaled = MinMaxScaler().fit_transform(x_train[potential])
        x_val_scaled = MinMaxScaler().fit_transform(x_val[potential])


        lr_model =  Ridge(alpha=0)
        lr_model.fit(x_train_scaled, y_train)


        lr_val_preds = lr_model.predict(x_val_scaled)
        lr_val_preds.clip(0,20,out=lr_val_preds)
        rms = math.sqrt(mean_squared_error(y_val, lr_val_preds))

        res[feat] = rms
        
    sorted_by_value = sorted(res.items(), key=lambda kv: kv[1])
    best_score = sorted_by_value[0][1]
    best_feature = sorted_by_value[0][0]
    print("best_score", best_score)
    print("best_feature", best_feature)
    if best_score < best:
        best = best_score
        keep.append(best_feature)
        all_f.remove(best_feature)
    else:
        break
    
keep

['item_id_mean_encoding', 'shop_id_mean_encoding', 'item_category_id_mean_encoding', 'month_mean_encoding', 'shop_cat_mean_encoding', 'shop_item_mean_encoding', 'date_block_num_mean_encoding', 'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3', 'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3', 'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3', 'shop_block_units_lag_1', 'shop_block_mean_lag_1', 'shop_block_median_lag_1', 'shop_block_min_lag_1', 'shop_block_max_lag_1', 'shop_block_std_lag_1', 'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1', 'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1', 'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1', 'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1', 'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1', 'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1', 'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1', 'shop_share', 'item_share', 'shop_pred

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return

best_score 1.2213909415288133
best_feature item_id_mean_encoding
keep ['item_id_mean_encoding']


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were

...

best_score 1.0587356513519661
best_feature shop_item_block_mean_lag_1
keep ['item_id_mean_encoding', 'shop_cat_mean_encoding', 'shop_item_block_units_lag_1', 'shop_cat_pred', 'shop_cat_block_units_lag_1', 'cat_pred', 'comp2_lag_1', 'shop_cat_block_max_rolling_3', 'shop_block_mean_lag_1', 'item_share_block_lag_1', 'shop_item_block_mean_lag_1']


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataCo

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataCo

best_score 1.0587553332327528
best_feature shop_share_block_lag_1


['item_id_mean_encoding',
 'shop_cat_mean_encoding',
 'shop_item_block_units_lag_1',
 'shop_cat_pred',
 'shop_cat_block_units_lag_1',
 'cat_pred',
 'comp2_lag_1',
 'shop_cat_block_max_rolling_3',
 'shop_block_mean_lag_1',
 'item_share_block_lag_1',
 'shop_item_block_mean_lag_1']

In [ ]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_by_value = sorted(x.items(), key=lambda kv: kv[1])[::-1]
sorted_by_value

In [91]:
gc.collect()
lgtrain = lgbm.Dataset(x_train[features], label=y_train)
lgval = lgbm.Dataset(x_val[features], label=y_val)


params = {
        "num_threads": 16,
        #"device": "gpu",
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        "max_bin": 30,#default 255
        #"num_leaves": 70, #default 31
        "bagging_fraction": 0.5,
        "bagging_freq": 1,
        "min_data_in_leaf": 2500,
        "feature_fraction": 0.7,
        #"lambda_l2": 3,
        #"max_depth": 7,
        #"min_gain_to_split": 10,
        "learning_rate" : 0.1,
        #"histogram_pool_size": 1000,
        #"categorical_column": [0,1,2,3,4]
}

evals_result = {}
lg_model = lgbm.train(params, lgtrain, 20000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=10, 
                      verbose_eval=10, 
                      evals_result=evals_result)

scores = {}
for i,score in enumerate(lg_model.feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's rmse: 1.16893
[20]	valid_0's rmse: 1.12071
[30]	valid_0's rmse: 1.10703
[40]	valid_0's rmse: 1.10052
[50]	valid_0's rmse: 1.09479
[60]	valid_0's rmse: 1.08897
[70]	valid_0's rmse: 1.08548
[80]	valid_0's rmse: 1.08165
[90]	valid_0's rmse: 1.07856
[100]	valid_0's rmse: 1.07572
[110]	valid_0's rmse: 1.07395
[120]	valid_0's rmse: 1.0719
[130]	valid_0's rmse: 1.07138
Early stopping, best iteration is:
[123]	valid_0's rmse: 1.07102


[('shop_cat_mean_encoding', 579),
 ('shop_cat_pred', 538),
 ('item_id_mean_encoding', 532),
 ('cat_pred', 525),
 ('shop_block_mean_lag_1', 398),
 ('shop_item_block_units_lag_1', 360),
 ('shop_cat_block_max_rolling_3', 333),
 ('shop_cat_block_units_lag_1', 307),
 ('shop_item_block_mean_lag_1', 118),
 ('item_share_block_lag_1', 0),
 ('comp2_lag_1', 0)]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor()

neigh.fit(x_train_scaled, y_train) 